In [4]:
import os
import multiprocessing
import logging
from typing import Optional
import sys
import itertools

import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
import rioxarray
import xarray as xr

from open_gira.wind import (
    advective_vector, rotational_field, interpolate_track,
    power_law_scale_factors, empty_wind_da, WIND_COORDS, ENV_PRESSURE
)

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

/tmp/ipykernel_12929/2643041496.py:8: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [5]:
# wind speed altitudes

# gradient level clearly not realistic, but we vary it to fit our estimated wind
# speeds to observations (or to better model results)
# the 18m figure is as a result of minimising pixel-wise errors between this model
# and that used in Done et al. 2020 with a physical boundary layer
GRADIENT_LEVEL_METRES = 18
SURFACE_LEVEL_METRES = 10

In [6]:
# DEFINE FUNCTIONS
def cleanup(output_path: str, downscale_factors_plot_path: str):
    """
    If we don't have a network, or tracks and we can't continue, write empty
    files and quit.
    """
    empty_wind_da().to_netcdf(output_path)
    os.system(f"touch {downscale_factors_plot_path}")
    sys.exit(0)

def process_track(
    track: pd.core.groupby.generic.DataFrameGroupBy,
    longitude: np.ndarray,
    latitude: np.ndarray,
    downscaling_factors: np.ndarray
) -> tuple[str, np.ndarray]:
    """
    Interpolate a track, reconstruct the advective and rotational vector wind
    fields, sum them and take the maximum of the wind vector magnitude across
    time. Optionally plot the wind fields and save to disk.

    Args:
        track: Subset of DataFrame describing a track. Must have a temporal
            index and the following fields: `min_pressure_hpa`,
            `max_wind_speed_ms`, `radius_to_max_winds_km`.
        longitude: Longitude values to construct evaluation grid
        latitude: Latitude values to construct evaluation grid
        downscaling_factors: Factors to bring gradient-level winds to surface.
        plot_max_wind: Whether to plot max wind fields
        plot_animation: Whether to plot wind field evolution
        plot_dir: Where to save optional plots.

    Returns:
        str: Track identifier
        np.ndarray: 2D array of maximum wind speed experienced at each grid pixel
    """

    track_id, = set(track.track_id)
    print(f'Processing {track_id}')

    logging.info(track_id)

    grid_shape: tuple[int, int] = (len(latitude), len(longitude))

    # we can't calculate the advective component without at least two points
    if len(track) == 1:
        return track_id, np.zeros(grid_shape)

    # basin of first record for storm track (storm genesis for synthetic tracks)
    basin: str = track.iloc[0, track.columns.get_loc("basin_id")]

    # Debug
    print('interpolating track')

    # interpolate track (avoid 'doughnut effect' of wind field from infrequent eye observations)
    track: gpd.GeoDataFrame = interpolate_track(track)

    geod_wgs84: pyproj.Geod = pyproj.CRS("epsg:4326").get_geod()

    # forward azimuth angle and distances from track eye to next track eye
    advection_azimuth_deg, _, eye_step_distance_m = geod_wgs84.inv(
        track.geometry.x.iloc[:-1],
        track.geometry.y.iloc[:-1],
        track.geometry.x.iloc[1:],
        track.geometry.y.iloc[1:],
    )

    # Debug
    print('gapfilling')
    
    # gapfill last period/distance values with penultimate value
    period = track.index[1:] - track.index[:-1]
    period = period.append(period[-1:])
    eye_step_distance_m = [*eye_step_distance_m, eye_step_distance_m[-1]]
    track["advection_azimuth_deg"] = [*advection_azimuth_deg, advection_azimuth_deg[-1]]
    
    # calculate eye speed
    track["eye_speed_ms"] = eye_step_distance_m / period.seconds.values

    # hemisphere belongs to {-1, 1}
    track["hemisphere"] = np.sign(track.geometry.y)

    adv_field: np.ndarray = np.zeros((len(track), *grid_shape), dtype=complex)
    rot_field: np.ndarray = np.zeros((len(track), *grid_shape), dtype=complex)

    # Debug
    print('Looping through track points')
    
    for track_i, track_point in enumerate(track.itertuples()):

        adv_vector: np.complex128 = advective_vector(
            track_point.advection_azimuth_deg,
            track_point.eye_speed_ms,
            track_point.hemisphere,
        )

        adv_field[track_i, :] = np.full(grid_shape, adv_vector)

        # maximum wind speed, less advective component
        # this is the maximum tangential wind speed in the eye's non-rotating reference frame
        max_wind_speed_relative_to_eye_ms: float = track_point.max_wind_speed_ms - np.abs(adv_vector)

        rot_field[track_i, :] = rotational_field(
            longitude,  # degrees
            latitude,  # degrees
            track_point.geometry.x,  # degrees
            track_point.geometry.y,  # degrees
            track_point.radius_to_max_winds_km * 1_000,  # convert to meters
            max_wind_speed_relative_to_eye_ms,
            track_point.min_pressure_hpa * 100,  # convert to Pascals
            ENV_PRESSURE[basin] * 100,  # convert to Pascals
        )

    # Debug
    print('Summing components of wind field')
    
    # sum components of wind field, (timesteps, y, x)
    wind_field: np.ndarray[complex] = adv_field + rot_field

    # take factors calculated from surface roughness of region and use to downscale speeds
    downscaled_wind_field = downscaling_factors * wind_field

    # Debug
    print('Finding vector magnitude')
    
    # find vector magnitude, then take max along timestep axis, giving (y, x)
    # N.B. np.max([np.nan, 1]) = np.nan, so use np.nanmax
    max_wind_speeds: np.ndarray[float] = np.nanmax(np.abs(downscaled_wind_field), axis=0)

    # Debug
    print(F'Finished {track_id}')
    
    return track_id, max_wind_speeds

In [7]:
# Define inputs (from Snakemake originally)
storm_file_path = '/home/mark/projects/open-gira/results/storm_tracks/STORM-constant/tracks_WP_1.geoparquet'
wind_grid_path = '/home/mark/projects/open-gira/results/direct/WP/wind_grid.tiff'
surface_roughness_path = '/home/mark/projects/open-gira/results/direct/WP/surface_roughness.tif'
# storm_set: set[str] = set(snakemake.params.storm_set)
# plot_max_wind: bool = snakemake.config["plot_wind"]["max_speed"]
# plot_animation: bool = snakemake.config["plot_wind"]["animation"]
n_proc = 1 # normally 4 DEBUG
output_path = '/home/mark/projects/open-gira/results/direct/WP/max_wind_field_WP.nc'

In [8]:
# Read tracks
tracks = gpd.read_parquet(storm_file_path)
if tracks.empty:
    print('No tracks found')

In [9]:
# filter tracks that don't make landfall
landfall_mask = tracks.groupby('track_id')['landfall'].any() # create a mask identifying track_ids that make landfall
landfall_tracks = landfall_mask[landfall_mask].index # filter the mask for only track_ids
tracks_filtered = tracks[tracks['track_id'].isin(landfall_tracks)]

In [10]:
# Group tracks
grouped_tracks = tracks_filtered.groupby('track_id')

In [11]:
# grid to evaluate wind speeds on, rioxarray will return midpoints of raster cells as dims
grid: xr.DataArray = rioxarray.open_rasterio(wind_grid_path)

In [12]:
# surface roughness raster for downscaling winds with
surface_roughness_raster: xr.DataArray = rioxarray.open_rasterio(surface_roughness_path)
# (1, y, x) where 1 is the number of surface roughness bands
# select the only value in the band dimension
surface_roughness: np.ndarray = surface_roughness_raster[0].values

In [13]:
# calculate factors to scale wind speeds from gradient-level to surface level,
# taking into account surface roughness as defined by the raster
downscaling_factors = power_law_scale_factors(
    surface_roughness,
    SURFACE_LEVEL_METRES,
    GRADIENT_LEVEL_METRES
)

In [14]:
# track is a tuple of track_id and the tracks subset, we only want the latter (create args for function)
args = ((track[1], grid.x, grid.y, downscaling_factors) for track in grouped_tracks)
# args = ((track[1], grid.x, grid.y, downscaling_factors) for track in itertools.islice(grouped_tracks,10))

In [15]:
# Temporary for only doing one iteration of wind field calc (change args to args_temp below)
args_temp = list(itertools.islice(args, 100))

In [ ]:
# Begin wind field estimation (revert to args from args_temp when not testing)
print('Estimating wind fields for %s storm tracks' % len(grouped_tracks))
max_wind_speeds: list[str, np.ndarray] = []
if n_proc > 1:
    with multiprocessing.Pool(processes=n_proc) as pool:
        max_wind_speeds = pool.starmap(process_track, args_temp)
else:
    for arg in args_temp:
        max_wind_speeds.append(process_track(*arg))

Estimating wind fields for 11588 storm tracks
Processing WP_0_0_1
interpolating track
gapfilling
Looping through track points
Summing components of wind field
Finding vector magnitude
Finished WP_0_0_1
Processing WP_0_0_14
interpolating track
gapfilling
Looping through track points
Summing components of wind field
Finding vector magnitude
Finished WP_0_0_14
Processing WP_0_0_18
interpolating track
gapfilling
Looping through track points
Summing components of wind field
Finding vector magnitude
Finished WP_0_0_18
Processing WP_0_0_19
interpolating track
gapfilling
Looping through track points
Summing components of wind field


In [ ]:
# sort by track_id so we have a reproducible order even after multiprocessing
max_wind_speeds = sorted(max_wind_speeds, key=lambda pair: pair[0])

In [ ]:
# Sacing maximum wind speeds to disk
track_ids, fields = zip(*max_wind_speeds)

In [ ]:
# write to disk as netCDF with CRS
da = xr.DataArray(
    data=np.stack(fields),
    dims=WIND_COORDS.keys(),
    coords=(
        ("event_id", list(track_ids)),
        ("latitude", grid.y.values),
        ("longitude", grid.x.values),
    ),
    attrs=dict(
        description="Maximum estimated wind speed during event",
        units="m s-1",
    ),
    name="max_wind_speed",
)
da = da.rio.write_crs("EPSG:4326")
encoding = {"max_wind_speed": {"zlib": True, "complevel": 9}}
da.to_netcdf(output_path, encoding=encoding)